* クロス集計する

In [ ]:
# !pip install japanize-matplotlib

In [ ]:
# !pip install git+https://github.com/darecophoenixx/wordroid.sblo.jp

In [ ]:
%matplotlib inline
from IPython.display import SVG, Image
from keras.utils.vis_utils import model_to_dot

import random, re

import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.mixture import GaussianMixture

import matplotlib.pyplot as plt
import seaborn as sns

# import japanize_matplotlib
# plt.rcParams['font.family']

# データ 読み込み

In [ ]:
'''元の０/１データ'''
df = pd.read_csv('../input/sample-data-wordanddoc2vec/actual_survey_data_jp.csv', index_col=0)
print(df.shape)
df.head()

In [ ]:
'''
因子負荷量
psych::faの方
'''
loadings = pd.read_csv('../input/apply-factanal-to-asd-calc-cor-from-col/res2_fa.csv', index_col=0)
print(loadings.shape)
loadings.head()

In [ ]:
'''疑似スコアをロード'''
df_scores = pd.read_csv('../input/asd-calc-cor-from-col-grouping-2/df_scores.csv', index_col=0)
print(df_scores.shape)
df_scores.head()

# クロス集計する

In [ ]:
cnt = []
for icol in loadings.columns.values:
    cnt.append((df_scores[icol+'cls'].values).sum())
cnt = pd.DataFrame(cnt, index=loadings.columns.values)
cnt

In [ ]:
%%time
ll = []
for ee in df.columns.values:
    ll2 = []
    for icol in loadings.columns.values:
        tmp = pd.crosstab(df_scores[icol+'cls']==1, df[ee], margins=True, normalize='index')
        ll2.append(tmp.loc[True].values)
    ll2.append(tmp.loc['All'].values)
    df_ll2 = pd.DataFrame(np.r_[ll2][:,1], index=loadings.columns.values.tolist()+['All'])
    ll.append(df_ll2)

res = pd.concat(ll, axis=1)
res.columns = df.columns
res

In [ ]:
res.iloc[:,:10]

In [ ]:
res.iloc[:,10:20]

In [ ]:
res.iloc[:,20:30]

In [ ]:
res.iloc[:,30:40]

In [ ]:
cnt
cnt1 = cnt.values
cnt1 = cnt1.flatten().tolist() + [df.shape[0]]
cnt1 = pd.DataFrame(cnt1)
cnt1.index = res.index.values
cnt1
res1 = pd.concat([res, pd.DataFrame(cnt1)], axis=1)
res1.columns.values[-1] = 'cnt'
res1

In [ ]:
res1.to_csv('crosstab.csv')